In [1]:
import re
import os
import json
import nltk
import scipy
import numpy as np
import pandas as pd

from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from itertools import chain
from tqdm import tqdm

C:\Users\erokhin_ai\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
diag = pd.read_csv('diag.csv', sep=';')
faq = pd.read_csv('faq.csv', sep=';')

C:\Users\erokhin_ai\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
diag = diag.fillna('')
faq = faq.fillna('')

In [5]:
tfidf = TfidfVectorizer(min_df = 2, max_df = 0.9)

In [6]:
tfidf.fit(list(diag.token_question) + list(diag.token_answer) + list(faq.question) + list(faq.answer))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
dists = cosine_similarity(tfidf.transform(diag.token_answer), tfidf.transform(faq.answer))

In [16]:
len(dists[dists>0.9])

216

In [18]:
def text_dist(text, text_list):
    sim = cosine_similarity(tfidf.transform(text), tfidf.transform(text_list))[0]
    return [text[0], text_list[sim==max(sim)].values[0], max(sim)]

In [22]:
sims = []
for i in tqdm(range(len(diag.token_answer))):
    sims.append(text_dist(([diag.token_answer[i]]), faq.answer))

100%|█████████████████████████████████████████████████████████████████████████| 171017/171017 [15:47<00:00, 180.49it/s]


In [24]:
sim_data = pd.DataFrame(sims, columns = ['raw_answer', 'faq_answer', 'distance'])

In [26]:
sim_data['raw_answer'] = [i[0] for i in sim_data.raw_answer]

In [30]:
sim_data.to_csv('answer_dists.csv',sep=';',index=False,encoding='utf8')

In [36]:
sims = []
for i in tqdm(range(len(diag.token_question))):
    sims.append(text_dist([diag.token_question[i]], faq.question))
    
sim_data = pd.DataFrame(sims, columns = ['raw_answer', 'faq_answer', 'distance'])

100%|█████████████████████████████████████████████████████████████████████████| 171017/171017 [09:14<00:00, 308.53it/s]


In [39]:
sim_data['raw_answer'] = [i[0] for i in sim_data.raw_answer]

In [42]:
sim_data[sim_data.distance>0.6]

,raw_answer,faq_answer,distance
144,здравствовать мочь ли я поменять пин код,мочь ли я поменять пин код свой карта bankname...,0.707174
197,как поменять пин код на карта,мочь ли я поменять пин код свой карта bankname...,0.611969
237,вчера создавать вклад в рубль сейчас хотеть по...,как можно пополнять вклад,0.624111
1123,какой процент,начисляться ли процент на процент,0.603319
1619,потерять карта,если я потерять карта что быть с деньги,0.749250
1908,где,где принимать карта,0.625610
2009,по номер карта,что происходить если я вносить сумма по номер ...,0.647429
2066,подсказывать начисляться ли процент на остаток...,начисляться ли процент на процент,0.601085
2477,добрый день взиматься ли плата за SMS банк при...,взиматься ли плата за услуга SMS банк,0.687033
3100,здраствовать хотеть узнавать какой число форми...,какой число у я быть формироваться выписка,0.681595
